In [1]:
import sys
import os
sys.path.append("..")
from knool.remote_sensing.satellite import tles_for_asat
from knool.geodata_processor import geo_info
import pandas as pd
from skyfield.api import load
from datetime import datetime as dt
from datetime import timedelta
from skyfield.api import utc
import numpy as np
from osgeo import gdal, osr, ogr

In [2]:
##READ tle file. https://qiita.com/grinpeaceman/items/22f06714a98dbfacce88

tles = load.tle_file("../knool/remote_sensing/data/two_line_elements/sat40697.txt") # TLE読み込み
satellite=tles[0]
sgp4_model = satellite.model

print("衛星名: ", satellite.name)
print("元期: ", satellite.epoch.utc_jpl())
print("衛星番号: ", sgp4_model.satnum)
print("エポック年: ", sgp4_model.epochyr)
print("エポック日: ", sgp4_model.epochdays)
print("エポックのユリウス日: ", sgp4_model.jdsatepoch)
print("ndot: ", sgp4_model.ndot)
print("nddot: ", sgp4_model.nddot)
print("弾道抗力係数 B*: ", sgp4_model.bstar)
print("軌道傾斜角[rad]: ", sgp4_model.inclo)
print("昇交点赤経[rad]: ", sgp4_model.nodeo)
print("離心率: ", sgp4_model.ecco)
print("近地点引数[rad]: ", sgp4_model.argpo)
print("平均近点角[rad]: ", sgp4_model.mo)
print("平均運動[rad/min]: ", sgp4_model.no_kozai)

衛星名:  None
元期:  A.D. 2015-Jun-23 03:50:23.3849 UTC
衛星番号:  40697
エポック年:  15
エポック日:  174.15999288
エポックのユリウス日:  2457196.5
ndot:  -1.333237623051224e-12
nddot:  0.0
弾道抗力係数 B*:  0.0
軌道傾斜角[rad]:  1.7206276070908537
昇交点赤経[rad]:  4.337586504835915
離心率:  7.98e-05
近地点引数[rad]:  4.953066393576204
平均近点角[rad]:  2.5043519997356394
平均運動[rad/min]:  0.06244205289471824


In [3]:
#get lat,lon,elevation from a tle
ts = load.timescale()  # skyfiled上で時刻を扱うモジュール
tle=tles[0]
date1 = dt(2018, 2, 1, 12, 15, 30, 2000, tzinfo=utc)
date1_sf=ts.from_datetime(date1)

geocentric=tle.at(date1_sf) #get xyz
subpoint = geocentric.subpoint()
lat = subpoint.latitude.degrees
lon = subpoint.longitude.degrees
ele = subpoint.elevation.m
print(geocentric.velocity)
print(geocentric.position)
print(geocentric.xyz)
print(lat,lon,ele)

[-0.00141461  0.00399833  0.00070632] au/day
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
-77.27650275778583 -162.68188667619526 815689.5183005631


In [4]:
#convert lla/ecef from ecef/lla using our package.
print(lat,lon,ele)
x,y,z=geo_info.convert_lla_to_ecef(lat,lon,ele)
print(x,y,z) # meter
lat,lon,ele=geo_info.convert_ecef_to_lla(x,y,z)
print(lat,lon,ele)

-77.27650275778583 -162.68188667619526 815689.5183005631
-1516878.3505331452 -472981.10875354183 -6995304.199162436
-77.27650276073706 -162.68188667619526 815689.5199458357


In [5]:
#create a kml file of a orbit line
infile=r"../knool/remote_sensing/data/two_line_elements/sat27424.txt"
test=tles_for_asat(infile)
test.set_period("2011-06-30 10:00:00","2011-07-02 11:40:00")
test.calc_positions_between("2011-07-01 10:00:00","2011-07-01 11:40:00",1)
test.export("../../test_data/output/test.kml")

/mnt/c/Users/Nakata Kazuki/Documents/プログラム/2022/kn-general_analysis_tools/kn-general_analysis_tools/notebook/../knool/remote_sensing/satellite.py:40: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(date), method='nearest')]]


In [6]:
#create a kml file of a orbit line
test.calc_positions_faster_between("2011-07-01 10:00:00","2011-07-01 11:40:00",1)
test.export("../../test_data/output/test2.kml")

/mnt/c/Users/Nakata Kazuki/Documents/プログラム/2022/kn-general_analysis_tools/kn-general_analysis_tools/notebook/../knool/remote_sensing/satellite.py:68: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(mdt), method='nearest')]]


In [7]:
#Creates a kml file of a polygon
test.calc_buff_area_between("2011-07-01 10:00:00","2011-07-01 11:40:00",1,1000000,ori="right")
test.export("../../test_data/output/test3.kml")

/mnt/c/Users/Nakata Kazuki/Documents/プログラム/2022/kn-general_analysis_tools/kn-general_analysis_tools/notebook/../knool/remote_sensing/satellite.py:112: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(mdt), method='nearest')]]


In [10]:
#Creates a kml file of a polygon
test.calc_scene_areas("2011-07-01 10:00:00","2011-07-01 11:40:00",1,1000000,2,ori="right")
test.export("../../test_data/output/test4.kml")

100 100 (100, 4, 2)


In [29]:
shpfile=r"../../test_data/matchup/AOI.shp"
result,result2 = test.calc_intersect_scene_areas("2011-07-01 0:00:00","2011-07-07 00:00:00",1,1450000,16,shpfile,ori="right")
print(len(result[0]))
test.export(r"../../test_data/output/test5.shp",option=0)
test.output=result2
test.export(r"../../test_data/output/test6.shp",option=0)


576 576 (576, 32, 2)
18


ERROR 1: TopologyException: Input geom 0 is invalid: Self-intersection at or near point -158.47777732024781 50.175298353999814 0 at -158.47777732024781 50.175298353999814 0
ERROR 1: TopologyException: Input geom 0 is invalid: Self-intersection at or near point 48.954272580095598 77.609960595231968 0 at 48.954272580095598 77.609960595231968 0
ERROR 1: TopologyException: Input geom 0 is invalid: Self-intersection at or near point 145.86949362608823 66.202165423884665 0 at 145.86949362608823 66.202165423884665 0
ERROR 1: TopologyException: Input geom 0 is invalid: Self-intersection at or near point 80.007895433903741 80.356767885044576 0 at 80.007895433903741 80.356767885044576 0
ERROR 1: TopologyException: Input geom 0 is invalid: Self-intersection at or near point -122.13347032323975 84.49852475589573 0 at -122.13347032323975 84.49852475589573 0
ERROR 1: TopologyException: Input geom 0 is invalid: Self-intersection at or near point 94.16195401236152 83.808240365857841 0 at 94.1619540123